<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/keras_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner -q

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

def model_builder(hp):
    """
    A function that builds a Keras model and defines the hyperparameter search space.
    """
    # Initialize a sequential model
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28))) # Input layer for 28x28 images

    # Tune the number of units in the first Dense layer.
    # hp.Int() defines an integer hyperparameter with a name, min value, max value, and step.
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    model.add(layers.Dense(10)) # Output layer for 10 classes

    # Tune the learning rate for the Adam optimizer.
    # hp.Choice() allows you to choose from a list of discrete values.
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Compile the model with the tunable learning rate
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

In [5]:
from warnings import filterwarnings
filterwarnings('ignore')

# Create a tuner instance
tuner = kt.RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=5, # Try 5 different combinations
    executions_per_trial=3, # Train each model 3 times
    directory='my_dir',
    project_name='intro_to_kt'
)

# Prepare data (e.g., using a dataset like MNIST)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_val, y_val = x_test[:5000], y_test[:5000]
x_test, y_test = x_test[5000:], y_test[5000:]

# Start the search
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

Trial 5 Complete [00h 08m 38s]
val_accuracy: 0.9697333375612894

Best val_accuracy So Far: 0.9738666613896688
Total elapsed time: 00h 27m 31s


In [6]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"The best number of units in the first layer is {best_hps.get('units')}.")
print(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

# Build and train the final model with the best hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

The best number of units in the first layer is 192.
The optimal learning rate for the optimizer is 0.001.
Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.9959 - loss: 0.0143 - val_accuracy: 0.9876 - val_loss: 0.0460
Epoch 2/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.9968 - loss: 0.0108 - val_accuracy: 0.9864 - val_loss: 0.0497
Epoch 3/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9982 - loss: 0.0077 - val_accuracy: 0.9874 - val_loss: 0.0490
Epoch 4/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9973 - loss: 0.0084 - val_accuracy: 0.9882 - val_loss: 0.0434
Epoch 5/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9982 - loss: 0.0061 - val_accuracy: 0.9882 - val_loss: 0.0576
Epoch 6/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9978 - loss: 0.0067 - val_accuracy: 0.9854 - val_loss: 0.0623
Epoch 7/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9988 - loss: 0.0053 - val_accuracy: 0.989